# AutoShorts 모듈별 테스트 노트북

각 Agent를 개별적으로 테스트하고 프롬프트를 확인할 수 있습니다.

## 📋 필요한 API 키

`.env` 파일에 다음 키들을 설정하세요:

```bash
# 필수
OPENAI_API_KEY=sk-...           # GPT-4o-mini (플롯 생성)
REDIS_URL=redis://localhost:6379

# 선택 (없으면 Stub 모드로 동작)
COMFYUI_URL=http://localhost:8188  # ComfyUI (이미지 생성)
MUBERT_LICENSE=...              # Mubert (음악 생성)
ELEVENLABS_API_KEY=...          # ElevenLabs (음성 생성)
PLAYHT_USER_ID=...              # PlayHT (음성 생성 대체)
PLAYHT_API_KEY=...
```

## 🚀 실행 전 체크리스트

1. Redis 서버 실행: `redis-server`
2. ComfyUI 실행 (선택): `cd ComfyUI && python main.py`
3. 가상환경 활성화: `source kvenv/bin/activate`
4. 필요한 패키지 설치 확인: `pip install -r backend/requirements.txt`

## 0. 환경 설정

In [ ]:
import sys
import os
from pathlib import Path
import json
from datetime import datetime

# 프로젝트 루트를 Python path에 추가
project_root = Path.cwd()
backend_path = project_root / "backend"
sys.path.insert(0, str(backend_path))

# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 환경 설정 완료")
print(f"프로젝트 루트: {project_root}")
print(f"Backend 경로: {backend_path}")

In [ ]:
# API 키 확인
from app.config import settings

print("=" * 50)
print("API 키 상태 확인")
print("=" * 50)

api_status = {
    "OpenAI (플롯 생성)": "✅" if settings.OPENAI_API_KEY else "❌",
    "ComfyUI (이미지)": "✅" if settings.COMFYUI_URL else "⚠️  Stub 모드",
    "Mubert (음악)": "✅" if settings.MUBERT_LICENSE else "⚠️  Stub 모드",
    "ElevenLabs (음성)": "✅" if settings.ELEVENLABS_API_KEY else "⚠️  Stub 모드",
    "PlayHT (음성 대체)": "✅" if settings.PLAYHT_API_KEY else "⚠️  Stub 모드",
    "Redis": "✅" if settings.REDIS_URL else "❌",
}

for service, status in api_status.items():
    print(f"{service:25s}: {status}")

print("\n⚠️  Stub 모드: API 키가 없으면 더미 데이터 생성")

## 1. 플롯 생성 테스트 (CSV → JSON)

GPT-4o-mini를 사용하여 시나리오 CSV를 생성하고 JSON으로 변환합니다.

In [ ]:
# 테스트 파라미터 설정
test_prompt = "우주를 여행하는 용감한 고양이의 모험"
num_characters = 2
num_cuts = 5
mode = "story"

# Run ID 생성 (실제와 동일한 형식)
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
prompt_clean = "".join(c for c in test_prompt if not c.isspace())[:8]
run_id = f"{timestamp}_{prompt_clean}"

print(f"Run ID: {run_id}")
print(f"프롬프트: {test_prompt}")
print(f"캐릭터 수: {num_characters}")
print(f"장면 수: {num_cuts}")
print(f"모드: {mode}")

In [ ]:
# CSV 생성
from app.utils.csv_to_json import generate_csv_from_prompt

print("=" * 50)
print("GPT-4o-mini에게 전달되는 프롬프트")
print("=" * 50)

# 시스템 프롬프트 출력 (실제 코드에서 사용되는 것과 동일)
system_prompt = f"""당신은 숏폼 영상 콘텐츠 시나리오 작가입니다.
사용자의 요청을 {num_cuts}개 장면으로 나누어 {'스토리' if mode == 'story' else '광고 콘텐츠'}를 만들어주세요.
등장인물은 {num_characters}명입니다.

각 장면마다 다음을 포함해야 합니다:
- scene_id: scene_1, scene_2, ... 형식 (이 필드로 장면 순서가 결정됨)
- char_id: char_1, char_2 (등장인물 ID)
- char_name: 캐릭터 이름 (창의적으로)
- text: 대사 또는 해설 내용 (큰따옴표 없이 텍스트만 입력)
- text_type: dialogue (대사) 또는 narration (해설)
- emotion: neutral, happy, sad, excited, angry, surprised 중 하나
- subtitle_position: top 또는 bottom
- duration_ms: 장면 지속시간 (보통 4000-6000)

**중요**:
- 반드시 CSV 형식으로만 출력하세요. 헤더와 데이터만 포함하고 다른 설명은 넣지 마세요.
- text 필드에는 큰따옴표를 넣지 마세요 (렌더링 시 자동 추가됨)
- text_type이 dialogue면 캐릭터 대사, narration이면 해설/나레이션입니다

CSV 형식:
scene_id,char_id,char_name,text,text_type,emotion,subtitle_position,duration_ms"""

print(system_prompt)
print("\n" + "=" * 50)
print(f"사용자 프롬프트: {test_prompt}")
print("=" * 50)

# CSV 생성 실행
print("\n🔄 CSV 생성 중...")
csv_path = generate_csv_from_prompt(
    run_id=run_id,
    prompt=test_prompt,
    num_characters=num_characters,
    num_cuts=num_cuts,
    mode=mode
)

print(f"\n✅ CSV 생성 완료: {csv_path}")

In [ ]:
# 생성된 CSV 확인
import pandas as pd

print("=" * 50)
print("생성된 CSV 내용")
print("=" * 50)

df = pd.read_csv(csv_path)
print(df.to_string())

print(f"\n총 {len(df)}개 행 생성됨")

In [ ]:
# CSV → JSON 변환
from app.utils.csv_to_json import csv_to_json

print("🔄 CSV → JSON 변환 중...")

json_path = csv_to_json(
    csv_path=str(csv_path),
    run_id=run_id,
    art_style="파스텔 수채화",
    music_genre="ambient"
)

print(f"✅ JSON 생성 완료: {json_path}")

In [ ]:
# JSON 내용 확인
with open(json_path, "r", encoding="utf-8") as f:
    layout = json.load(f)

print("=" * 50)
print("JSON 레이아웃 구조")
print("=" * 50)

print(f"\nProject ID: {layout['project_id']}")
print(f"Title: {layout['title']}")
print(f"Total Duration: {layout['timeline']['total_duration_ms']}ms")

print(f"\n캐릭터 ({len(layout['characters'])}명):")
for char in layout['characters']:
    print(f"  - {char['name']} (ID: {char['char_id']}, Seed: {char['seed']})")

print(f"\n씬 ({len(layout['scenes'])}개):")
for scene in layout['scenes']:
    print(f"\n  [{scene['scene_id']}] ({scene['duration_ms']}ms)")
    print(f"    이미지 슬롯: {len(scene['images'])}개")
    print(f"    대사: {len(scene['dialogue'])}개")
    print(f"    자막: {len(scene['subtitles'])}개")
    
    # 자막 내용 확인 (text_type에 따른 큰따옴표 추가 확인)
    if scene['subtitles']:
        for subtitle in scene['subtitles']:
            print(f"      자막 텍스트: {subtitle['text']}")

print("\n" + "=" * 50)
print("✅ text_type이 dialogue인 경우 큰따옴표가 자동 추가됨")
print("=" * 50)

## 2. 이미지 생성 테스트 (ComfyUI)

ComfyUI를 사용하여 캐릭터/배경 이미지를 생성합니다.

In [ ]:
# ComfyUI 클라이언트 테스트
from app.config import settings

if settings.COMFYUI_URL:
    from app.providers.images.comfyui_client import ComfyUIClient
    image_client = ComfyUIClient()
    print(f"✅ ComfyUI 클라이언트 초기화 완료: {settings.COMFYUI_URL}")
else:
    from app.providers.images.stub_client import StubImageClient
    image_client = StubImageClient()
    print("⚠️  Stub 모드로 실행 (ComfyUI 없음)")

In [ ]:
# 첫 번째 씬의 이미지 생성 테스트
scene = layout['scenes'][0]
char = layout['characters'][0]

print("=" * 50)
print("이미지 생성 프롬프트")
print("=" * 50)

# 프롬프트 구성
art_style = layout['metadata']['art_style']
persona = char['persona']
char_seed = char['seed']
bg_seed = scene['bg_seed']

image_prompt = f"{art_style}, {persona}, standing, neutral expression"

print(f"캐릭터: {char['name']}")
print(f"Persona: {persona}")
print(f"Character Seed: {char_seed}")
print(f"Background Seed: {bg_seed}")
print(f"\n최종 프롬프트: {image_prompt}")
print("\nNegative Prompt: blurry, low quality, distorted")

print("\n🔄 이미지 생성 중...")

try:
    image_path = image_client.generate(
        prompt=image_prompt,
        negative_prompt="blurry, low quality, distorted",
        width=1080,
        height=1920,
        seed=char_seed,
        output_path=f"app/data/outputs/{run_id}/scene_1_center.png"
    )
    
    print(f"\n✅ 이미지 생성 완료: {image_path}")
    
    # 이미지 표시 (Jupyter에서)
    from IPython.display import Image, display
    if Path(image_path).exists():
        display(Image(filename=image_path, width=300))
        
except Exception as e:
    print(f"\n❌ 이미지 생성 실패: {e}")
    print("\n💡 ComfyUI가 실행 중인지 확인하세요: http://localhost:8188")

## 3. 음악 생성 테스트 (Mubert)

Mubert API를 사용하여 배경음악을 생성합니다.

In [ ]:
# Mubert 클라이언트 테스트
from app.config import settings

if settings.MUBERT_LICENSE:
    from app.providers.music.mubert_client import MubertClient
    music_client = MubertClient()
    print("✅ Mubert 클라이언트 초기화 완료")
else:
    from app.providers.music.stub_client import StubMusicClient
    music_client = StubMusicClient()
    print("⚠️  Stub 모드로 실행 (Mubert 없음)")

In [ ]:
# 배경음악 생성
genre = layout['metadata']['music_genre']
duration_sec = layout['timeline']['total_duration_ms'] / 1000

print("=" * 50)
print("음악 생성 파라미터")
print("=" * 50)
print(f"장르: {genre}")
print(f"길이: {duration_sec}초")
print(f"무드: cinematic, dreamy")

print("\n🔄 음악 생성 중...")

try:
    audio_path = music_client.generate(
        genre=genre,
        mood="cinematic,dreamy",
        duration_sec=int(duration_sec),
        output_path=f"app/data/outputs/{run_id}/global_bgm.mp3"
    )
    
    print(f"\n✅ 음악 생성 완료: {audio_path}")
    
    # 오디오 재생 (Jupyter에서)
    from IPython.display import Audio, display
    if Path(audio_path).exists():
        display(Audio(filename=audio_path))
        
except Exception as e:
    print(f"\n❌ 음악 생성 실패: {e}")

## 4. 음성 생성 테스트 (ElevenLabs / PlayHT)

TTS API를 사용하여 캐릭터 대사를 생성합니다.

In [ ]:
# TTS 클라이언트 선택
from app.config import settings

if settings.ELEVENLABS_API_KEY:
    from app.providers.tts.elevenlabs_client import ElevenLabsClient
    tts_client = ElevenLabsClient()
    print("✅ ElevenLabs 클라이언트 초기화 완료")
elif settings.PLAYHT_API_KEY:
    from app.providers.tts.playht_client import PlayHTClient
    tts_client = PlayHTClient()
    print("✅ PlayHT 클라이언트 초기화 완료")
else:
    from app.providers.tts.stub_client import StubTTSClient
    tts_client = StubTTSClient()
    print("⚠️  Stub 모드로 실행 (TTS 없음)")

In [ ]:
# 첫 번째 대사 생성
dialogue = scene['dialogue'][0]

print("=" * 50)
print("음성 생성 파라미터")
print("=" * 50)
print(f"캐릭터: {char['name']}")
print(f"대사: {dialogue['text']}")
print(f"감정: {dialogue['emotion']}")
print(f"Voice ID: {char.get('voice_profile', 'default')}")

print("\n🔄 음성 생성 중...")

try:
    voice_path = tts_client.generate(
        text=dialogue['text'],
        voice_id=char.get('voice_profile', 'default'),
        emotion=dialogue['emotion'],
        output_path=f"app/data/outputs/{run_id}/scene_1_line_1.mp3"
    )
    
    print(f"\n✅ 음성 생성 완료: {voice_path}")
    
    # 오디오 재생
    from IPython.display import Audio, display
    if Path(voice_path).exists():
        display(Audio(filename=voice_path))
        
except Exception as e:
    print(f"\n❌ 음성 생성 실패: {e}")

## 5. 통합 테스트 - 전체 씬 생성

모든 씬의 에셋을 생성하고 JSON을 업데이트합니다.

In [ ]:
# 전체 씬 에셋 생성
print("=" * 50)
print(f"전체 {len(layout['scenes'])}개 씬 에셋 생성")
print("=" * 50)

for scene_idx, scene in enumerate(layout['scenes']):
    scene_id = scene['scene_id']
    print(f"\n[{scene_id}] 처리 중...")
    
    # 이미지 생성
    for img_slot in scene['images']:
        if img_slot['type'] == 'character':
            char = next(c for c in layout['characters'] if c['char_id'] == img_slot['ref_id'])
            prompt = f"{art_style}, {char['persona']}, standing"
            
            try:
                img_path = image_client.generate(
                    prompt=prompt,
                    negative_prompt="blurry, low quality",
                    width=1080,
                    height=1920,
                    seed=char['seed'],
                    output_path=f"app/data/outputs/{run_id}/{scene_id}_{img_slot['slot_id']}.png"
                )
                img_slot['image_url'] = str(img_path)
                print(f"  ✅ 이미지: {img_path}")
            except Exception as e:
                print(f"  ❌ 이미지 생성 실패: {e}")
    
    # 음성 생성
    for dialogue_idx, dialogue in enumerate(scene['dialogue']):
        char = next(c for c in layout['characters'] if c['char_id'] == dialogue['char_id'])
        
        try:
            voice_path = tts_client.generate(
                text=dialogue['text'],
                voice_id=char.get('voice_profile', 'default'),
                emotion=dialogue['emotion'],
                output_path=f"app/data/outputs/{run_id}/{dialogue['line_id']}.mp3"
            )
            dialogue['audio_url'] = str(voice_path)
            print(f"  ✅ 음성: {voice_path}")
        except Exception as e:
            print(f"  ❌ 음성 생성 실패: {e}")

print("\n" + "=" * 50)
print("✅ 전체 에셋 생성 완료")
print("=" * 50)

In [ ]:
# 업데이트된 JSON 저장
updated_json_path = Path(json_path).parent / "layout_with_assets.json"
with open(updated_json_path, "w", encoding="utf-8") as f:
    json.dump(layout, f, indent=2, ensure_ascii=False)

print(f"✅ 업데이트된 JSON 저장: {updated_json_path}")

# 최종 결과 요약
print("\n" + "=" * 50)
print("최종 결과 요약")
print("=" * 50)

total_images = sum(len(s['images']) for s in layout['scenes'])
total_voices = sum(len(s['dialogue']) for s in layout['scenes'])

print(f"생성된 이미지: {total_images}개")
print(f"생성된 음성: {total_voices}개")
print(f"배경음악: 1개")
print(f"\n모든 에셋 저장 위치: app/data/outputs/{run_id}/")

## 6. 프롬프트 분석 및 최적화

생성된 프롬프트를 분석하고 개선 방안을 제안합니다.

In [ ]:
# 프롬프트 분석
print("=" * 50)
print("프롬프트 분석")
print("=" * 50)

print("\n1. GPT 시스템 프롬프트 (플롯 생성)")
print("-" * 50)
print(f"길이: {len(system_prompt)} 문자")
print(f"주요 지시사항: CSV 형식, text_type 필드, 큰따옴표 금지")

print("\n2. 이미지 생성 프롬프트")
print("-" * 50)
for char in layout['characters']:
    prompt = f"{art_style}, {char['persona']}, standing, neutral expression"
    print(f"[{char['name']}] {prompt}")
    print(f"  길이: {len(prompt)} 문자")
    print(f"  Seed: {char['seed']}")

print("\n3. 음성 생성 파라미터")
print("-" * 50)
for scene in layout['scenes']:
    for dialogue in scene['dialogue']:
        char = next(c for c in layout['characters'] if c['char_id'] == dialogue['char_id'])
        print(f"[{char['name']}] {dialogue['text'][:30]}...")
        print(f"  감정: {dialogue['emotion']}")
        print(f"  길이: {len(dialogue['text'])} 문자")

print("\n4. 음악 생성 파라미터")
print("-" * 50)
print(f"장르: {genre}")
print(f"길이: {duration_sec}초")
print(f"무드: cinematic, dreamy")

## 7. 정리 및 다음 단계

생성된 모든 파일과 다음 단계를 확인합니다.

In [ ]:
# 생성된 파일 목록
output_dir = Path(f"app/data/outputs/{run_id}")

print("=" * 50)
print(f"생성된 파일 목록 ({output_dir})")
print("=" * 50)

if output_dir.exists():
    files = sorted(output_dir.glob("*"))
    
    for file in files:
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"{file.name:40s} {size_mb:>8.2f} MB")
    
    print(f"\n총 {len(files)}개 파일")
else:
    print("⚠️  출력 디렉토리가 없습니다.")

print("\n" + "=" * 50)
print("다음 단계")
print("=" * 50)
print("1. Director Agent로 영상 합성 (MoviePy)")
print("2. QA Agent로 품질 검수")
print("3. 최종 영상 확인")
print("\n💡 전체 파이프라인 실행은 FastAPI + Celery로 진행하세요.")